In [1]:
import sys

import gym
import numpy as np
import scipy.integrate as sc_integrate

from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import A2C, DQN

# using wredsen's symbtools fork (https://github.com/wredsen/symbtools @ DAE_statefeedback), assuming repos SA-Wrede and symbtools share the same parent directory
sys.path.append('../../symbtools/')
import symbtools as st
import sympy as sp
import pickle

## Mathematical system description with SymPy / symbtools

In [2]:
Np = 1 # number of passive coordinates (p = phi1)
Nq = 1 # number of actuated coordinates (q = x0)
n = Np + Nq
pp = st.symb_vector("p1:{0}".format(Np+1))
qq = st.symb_vector("q1:{0}".format(Nq+1))

# all coordinates and their derivatives
ttheta = st.row_stack(pp, qq) ##:T
tthetad = st.time_deriv(ttheta, ttheta) ##:T
tthetadd = st.time_deriv(ttheta, ttheta, order=2) ##:T
st.make_global(ttheta, tthetad)

F = sp.Symbol('F')

params = sp.symbols('m0, m1, l1, g, d0, d1')
st.make_global(params)
params_values = [(m0, 1.0), (m1, 0.1), (l1, 0.5), (g, 9.81),
                 (d0, 0.01), (d1, 0.01)]

In [3]:
# load model etc. from pickle of flatness analysis notebook
with open("../single_crane_notebooks/model_single_crane.pcl", "rb") as pfile:
    mod = pickle.load(pfile) # mod instead of data
    #locals().update(data)

In [4]:
mod.xx

Matrix([
[   p1],
[   q1],
[pdot1],
[qdot1]])

In [5]:
mod.calc_state_eq(force_recalculation=True)
mod.eqns

Matrix([
[            d0*pdot1 + g*l1*m1*sin(p1) + l1**2*m1*pddot1 - l1*m1*qddot1*cos(p1)],
[-Fcomp + 1.0*m0*qddot1 + m1*(-l1*pddot1*cos(p1) + l1*pdot1**2*sin(p1) + qddot1)]])

In [6]:
states_dot = mod.f + mod.g * F

In [7]:
states_dot_wo_params = states_dot.subs(params_values)

In [8]:
states_dot_func = st.expr_to_func([*mod.xx, F], states_dot_wo_params)

## Utility functions for Gym environment

In [9]:
"""
Utility functions used for classic control environments.
"""

from typing import Optional, SupportsFloat, Tuple


def verify_number_and_cast(x: SupportsFloat) -> float:
    """Verify parameter is a single number and cast to a float."""
    try:
        x = float(x)
    except (ValueError, TypeError):
        raise ValueError(f"An option ({x}) could not be converted to a float.")
    return x


def maybe_parse_reset_bounds(
    options: Optional[dict], default_low: float, default_high: float
) -> Tuple[float, float]:
    """
    This function can be called during a reset() to customize the sampling
    ranges for setting the initial state distributions.

    Args:
      options: Options passed in to reset().
      default_low: Default lower limit to use, if none specified in options.
      default_high: Default upper limit to use, if none specified in options.

    Returns:
      Tuple of the lower and upper limits.
    """
    if options is None:
        return default_low, default_high

    low = options.get("low") if "low" in options else default_low
    high = options.get("high") if "high" in options else default_high

    # We expect only numerical inputs.
    low = verify_number_and_cast(low)
    high = verify_number_and_cast(high)
    if low > high:
        raise ValueError(
            f"Lower bound ({low}) must be lower than higher bound ({high})."
        )

    return low, high

In [10]:
"""A utility class to collect render frames from a function that computes a single frame."""
from typing import Any, Callable, List, Optional, Set

# list of modes with which render function returns None
NO_RETURNS_RENDER = {"human"}

# list of modes with which render returns just a single frame of the current state
SINGLE_RENDER = {"single_rgb_array", "single_depth_array", "single_state_pixels"}


class Renderer:
    """This class serves to easily integrate collection of renders for environments that can computes a single render.

    To use this function:
    - instantiate this class with the mode and the function that computes a single frame
    - call render_step method each time the frame should be saved in the list
      (usually at the end of the step and reset methods)
    - call get_renders whenever you want to retrieve renders
      (usually in the render method)
    - call reset to clean the render list
      (usually in the reset method of the environment)
    """

    def __init__(
        self,
        mode: Optional[str],
        render: Callable[[str], Any],
        no_returns_render: Optional[Set[str]] = None,
        single_render: Optional[Set[str]] = None,
    ):
        """Instantiates a Renderer object.

        Args:
            mode (Optional[str]): Way to render
            render (Callable[[str], Any]): Function that receives the mode and computes a single frame
            no_returns_render (Optional[Set[str]]): Set of render modes that don't return any value.
                The default value is the set {"human"}.
            single_render (Optional[Set[str]]): Set of render modes that should return a single frame.
                The default value is the set {"single_rgb_array", "single_depth_array", "single_state_pixels"}.
        """
        if no_returns_render is None:
            no_returns_render = NO_RETURNS_RENDER
        if single_render is None:
            single_render = SINGLE_RENDER

        self.no_returns_render = no_returns_render
        self.single_render = single_render
        self.mode = mode
        self.render = render
        self.render_list = []

    def render_step(self) -> None:
        """Computes a frame and save it to the render collection list.

        This method should be usually called inside environment's step and reset method.
        """
        if self.mode is not None and self.mode not in self.single_render:
            render_return = self.render(self.mode)
            if self.mode not in self.no_returns_render:
                self.render_list.append(render_return)

    def get_renders(self) -> Optional[List]:
        """Pops all the frames from the render collection list.

        This method should be usually called in the environment's render method to retrieve the frames collected till this time step.
        """
        if self.mode in self.single_render:
            return self.render(self.mode)
        elif self.mode is not None and self.mode not in self.no_returns_render:
            renders = self.render_list
            self.render_list = []
            return renders

    def reset(self):
        """Resets the render collection list.

        This method should be usually called inside environment's reset method.
        """
        self.render_list = []

## Gym environment for crane

In [11]:
"""
Classic cart-pole system implemented by Rich Sutton et al.
Copied from http://incompleteideas.net/sutton/book/code/pole.c
permalink: https://perma.cc/C9ZM-652R
"""
import math
from typing import Optional, Union

import numpy as np

import gym
from gym import logger, spaces
#from gym.envs.classic_control import utils
from gym.error import DependencyNotInstalled
#from gym.utils.renderer import Renderer


class CartPoleEnv(gym.Env):
    """
    ### Description

    This environment corresponds to the version of the cart-pole problem described by Barto, Sutton, and Anderson in
    ["Neuronlike Adaptive Elements That Can Solve Difficult Learning Control Problem"](https://ieeexplore.ieee.org/document/6313077).
    A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track.
    The pendulum is placed upright on the cart and the goal is to balance the pole by applying forces
     in the left and right direction on the cart.

    ### Action Space

    The action is a `ndarray` with shape `(1,)` which can take values `{0, 1}` indicating the direction
     of the fixed force the cart is pushed with.

    | Num | Action                 |
    |-----|------------------------|
    | 0   | Push cart to the left  |
    | 1   | Push cart to the right |

    **Note**: The velocity that is reduced or increased by the applied force is not fixed and it depends on the angle
     the pole is pointing. The center of gravity of the pole varies the amount of energy needed to move the cart underneath it

    ### Observation Space

    The observation is a `ndarray` with shape `(4,)` with the values corresponding to the following positions and velocities:

    | Num | Observation           | Min                 | Max               |
    |-----|-----------------------|---------------------|-------------------|
    | 0   | Cart Position         | -4.8                | 4.8               |
    | 1   | Cart Velocity         | -Inf                | Inf               |
    | 2   | Pole Angle            | ~ -0.418 rad (-24°) | ~ 0.418 rad (24°) |
    | 3   | Pole Angular Velocity | -Inf                | Inf               |

    **Note:** While the ranges above denote the possible values for observation space of each element,
        it is not reflective of the allowed values of the state space in an unterminated episode. Particularly:
    -  The cart x-position (index 0) can be take values between `(-4.8, 4.8)`, but the episode terminates
       if the cart leaves the `(-2.4, 2.4)` range.
    -  The pole angle can be observed between  `(-.418, .418)` radians (or **±24°**), but the episode terminates
       if the pole angle is not in the range `(-.2095, .2095)` (or **±12°**)

    ### Rewards

    Since the goal is to keep the pole upright for as long as possible, a reward of `+1` for every step taken,
    including the termination step, is allotted. The threshold for rewards is 475 for v1.

    ### Starting State

    All observations are assigned a uniformly random value in `(-0.05, 0.05)`

    ### Episode End

    The episode ends if any one of the following occurs:

    1. Termination: Pole Angle is greater than ±12°
    2. Termination: Cart Position is greater than ±2.4 (center of the cart reaches the edge of the display)
    3. Truncation: Episode length is greater than 500 (200 for v0)

    ### Arguments

    ```
    gym.make('CartPole-v1')
    ```

    No additional arguments are currently supported.
    """

    metadata = {
        "render_modes": ["human", "rgb_array", "single_rgb_array"],
        "render_fps": 50,
    }

    def __init__(self, render_mode: Optional[str] = None):
    
        # geometrics for rendering
        self.length = 0.5  # actually half the pole's length
        
        # magnitude for discrete force applied
        self.force_mag = 1.0
        
        #simulation step width
        self.deltaT = 0.02  # seconds between state updates

        # Angle at which to fail the episode
        self.theta_threshold_radians = 12 * 2 * math.pi / 360
        self.x_threshold = 2.4

        high_act = np.array(
            [ 
                1.0
            ],
            dtype=np.float32,
        )
        self.action_space = spaces.Box(-high_act, high_act, dtype=np.float32)
        
        # Angle limit set to 2 * theta_threshold_radians so failing observation
        # is still within bounds.
        high_obs = np.array(
            [
                self.theta_threshold_radians * 2,
                 self.x_threshold * 2,
                np.finfo(np.float32).max,
                np.finfo(np.float32).max,
            ],
            dtype=np.float32,
        )
        self.observation_space = spaces.Box(-high_obs, high_obs, dtype=np.float32)

        self.render_mode = render_mode
        self.renderer = Renderer(self.render_mode, self._render)

        self.screen_width = 600
        self.screen_height = 400
        self.screen = None
        self.clock = None
        self.isopen = True
        self.state = None

        self.steps_beyond_terminated = None

    def step(self, action):
        err_msg = f"{action!r} ({type(action)}) invalid"
        assert self.action_space.contains(action), err_msg
        assert self.state is not None, "Call reset before using step method."
        p1, q1, p1_dot, q1_dot = self.state
        force = self.force_mag * action
        
        states_dot_now = states_dot_func(*self.state, force)
        p1, q1, p1_dot, q1_dot = self.state + self.deltaT * states_dot_now
        
        self.state = (p1, q1, p1_dot, q1_dot)

        terminated = bool(
            q1 < -self.x_threshold
            or q1 > self.x_threshold
            or p1 < -self.theta_threshold_radians
            or p1 > self.theta_threshold_radians
        )

        if not terminated:
            reward = 1.0
        elif self.steps_beyond_terminated is None:
            # Pole just fell!
            self.steps_beyond_terminated = 0
            reward = 1.0
        else:
            if self.steps_beyond_terminated == 0:
                logger.warn(
                    "You are calling 'step()' even though this "
                    "environment has already returned terminated = True. You "
                    "should always call 'reset()' once you receive 'terminated = "
                    "True' -- any further steps are undefined behavior."
                )
            self.steps_beyond_terminated += 1
            reward = 0.0

        self.renderer.render_step()
        return np.array(self.state, dtype=np.float32), reward, terminated, {"info": False}#False, {}

    def reset(
        self,
        *,
        seed: Optional[int] = None,
        options: Optional[dict] = None,
    ):
        #super().reset()(seed=seed)
        # Note that if you use custom reset bounds, it may lead to out-of-bound
        # state/observations.
        low, high = maybe_parse_reset_bounds(
            options, -0.05, 0.05  # default low
        )  # default high
        self.state = np.random.uniform(low=low, high=high, size=(4,))
        self.steps_beyond_terminated = None
        self.renderer.reset()
        self.renderer.render_step()
        return np.array(self.state, dtype=np.float32)#, {}

    def render(self):
        return self.renderer.get_renders()

    def _render(self, mode="human"):
        assert mode in self.metadata["render_modes"]
        try:
            import pygame
            from pygame import gfxdraw
        except ImportError:
            raise DependencyNotInstalled(
                "pygame is not installed, run `pip install gym[classic_control]`"
            )

        if self.screen is None:
            pygame.init()
            if mode == "human":
                pygame.display.init()
                self.screen = pygame.display.set_mode(
                    (self.screen_width, self.screen_height)
                )
            else:  # mode in {"rgb_array", "single_rgb_array"}
                self.screen = pygame.Surface((self.screen_width, self.screen_height))
        if self.clock is None:
            self.clock = pygame.time.Clock()

        world_width = self.x_threshold * 2
        scale = self.screen_width / world_width
        polewidth = 10.0
        polelen = scale * (2 * self.length)
        cartwidth = 50.0
        cartheight = 30.0

        if self.state is None:
            return None

        x = self.state

        self.surf = pygame.Surface((self.screen_width, self.screen_height))
        self.surf.fill((255, 255, 255))

        l, r, t, b = -cartwidth / 2, cartwidth / 2, cartheight / 2, -cartheight / 2
        axleoffset = cartheight / 4.0
        cartx = x[1] * scale + self.screen_width / 2.0  # MIDDLE OF CART
        carty = 100  # TOP OF CART
        cart_coords = [(l, b), (l, t), (r, t), (r, b)]
        cart_coords = [(c[0] + cartx, c[1] + carty) for c in cart_coords]
        gfxdraw.aapolygon(self.surf, cart_coords, (0, 0, 0))
        gfxdraw.filled_polygon(self.surf, cart_coords, (0, 0, 0))

        l, r, t, b = (
            -polewidth / 2,
            polewidth / 2,
            polelen - polewidth / 2,
            -polewidth / 2,
        )

        pole_coords = []
        for coord in [(l, b), (l, t), (r, t), (r, b)]:
            coord = pygame.math.Vector2(coord).rotate_rad(-x[0])
            coord = (coord[0] + cartx, coord[1] + carty + axleoffset)
            pole_coords.append(coord)
        gfxdraw.aapolygon(self.surf, pole_coords, (202, 152, 101))
        gfxdraw.filled_polygon(self.surf, pole_coords, (202, 152, 101))

        gfxdraw.aacircle(
            self.surf,
            int(cartx),
            int(carty + axleoffset),
            int(polewidth / 2),
            (129, 132, 203),
        )
        gfxdraw.filled_circle(
            self.surf,
            int(cartx),
            int(carty + axleoffset),
            int(polewidth / 2),
            (129, 132, 203),
        )

        gfxdraw.hline(self.surf, 0, self.screen_width, carty, (0, 0, 0))

        self.surf = pygame.transform.flip(self.surf, False, True)
        self.screen.blit(self.surf, (0, 0))
        if mode == "human":
            pygame.event.pump()
            self.clock.tick(self.metadata["render_fps"])
            pygame.display.flip()

        elif mode in {"rgb_array", "single_rgb_array"}:
            return np.transpose(
                np.array(pygame.surfarray.pixels3d(self.screen)), axes=(1, 0, 2)
            )

    def close(self):
        if self.screen is not None:
            import pygame

            pygame.display.quit()
            pygame.quit()
            self.isopen = False

In [18]:
# environment without renderings for training
env = CartPoleEnv()
# environment with renderings for validating
env_rendering = CartPoleEnv(render_mode = "human")

In [19]:
env.reset()
env.step(env.action_space.sample())
env.observation_space
check_env(env)

## Learning the model

In [14]:
%%time
# Learning!
model = A2C('MlpPolicy', env, verbose=1, tensorboard_log="./a2c_cartpole_tensorboard/")
model.learn(total_timesteps=100000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


2022-09-06 13:46:59.267779: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/wredi/Desktop/FraunhoferIIS/OMSysIdent/install/linux/lib:
2022-09-06 13:46:59.267801: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Logging to ./a2c_cartpole_tensorboard/A2C_8
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 221      |
|    ep_rew_mean        | 221      |
| time/                 |          |
|    fps                | 209      |
|    iterations         | 100      |
|    time_elapsed       | 2        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.42    |
|    explained_variance | -0.107   |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 3.06     |
|    std                | 0.996    |
|    value_loss         | 8.18     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 448      |
|    ep_rew_mean        | 448      |
| time/                 |          |
|    fps                | 306      |
|    iterations         | 200      |
|    time_elapsed       | 3    

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 440       |
|    ep_rew_mean        | 440       |
| time/                 |           |
|    fps                | 512       |
|    iterations         | 1400      |
|    time_elapsed       | 13        |
|    total_timesteps    | 7000      |
| train/                |           |
|    entropy_loss       | -1.41     |
|    explained_variance | -4.15e-05 |
|    learning_rate      | 0.0007    |
|    n_updates          | 1399      |
|    policy_loss        | 0.757     |
|    std                | 0.992     |
|    value_loss         | 0.539     |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 440       |
|    ep_rew_mean        | 440       |
| time/                 |           |
|    fps                | 516       |
|    iterations         | 1500      |
|    time_elapsed       | 14        |
|    total_t

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 586      |
|    ep_rew_mean        | 586      |
| time/                 |          |
|    fps                | 541      |
|    iterations         | 2700     |
|    time_elapsed       | 24       |
|    total_timesteps    | 13500    |
| train/                |          |
|    entropy_loss       | -1.39    |
|    explained_variance | -0.72    |
|    learning_rate      | 0.0007   |
|    n_updates          | 2699     |
|    policy_loss        | 3.11e-05 |
|    std                | 0.973    |
|    value_loss         | 6.4e-10  |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 586      |
|    ep_rew_mean        | 586      |
| time/                 |          |
|    fps                | 542      |
|    iterations         | 2800     |
|    time_elapsed       | 25       |
|    total_timesteps    | 14000    |
|

...

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 1.1e+03   |
|    ep_rew_mean        | 1.1e+03   |
| time/                 |           |
|    fps                | 573       |
|    iterations         | 14400     |
|    time_elapsed       | 125       |
|    total_timesteps    | 72000     |
| train/                |           |
|    entropy_loss       | -1.34     |
|    explained_variance | 0.00189   |
|    learning_rate      | 0.0007    |
|    n_updates          | 14399     |
|    policy_loss        | -0.000702 |
|    std                | 0.925     |
|    value_loss         | 2.48e-07  |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 1.1e+03   |
|    ep_rew_mean        | 1.1e+03   |
| time/                 |           |
|    fps                | 573       |
|    iterations         | 14500     |
|    time_elapsed       | 126       |
|    total_t

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.1e+03  |
|    ep_rew_mean        | 1.1e+03  |
| time/                 |          |
|    fps                | 573      |
|    iterations         | 15700    |
|    time_elapsed       | 136      |
|    total_timesteps    | 78500    |
| train/                |          |
|    entropy_loss       | -1.37    |
|    explained_variance | 0.0184   |
|    learning_rate      | 0.0007   |
|    n_updates          | 15699    |
|    policy_loss        | -0.00081 |
|    std                | 0.951    |
|    value_loss         | 3.07e-07 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.1e+03  |
|    ep_rew_mean        | 1.1e+03  |
| time/                 |          |
|    fps                | 573      |
|    iterations         | 15800    |
|    time_elapsed       | 137      |
|    total_timesteps    | 79000    |
|

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 1.1e+03   |
|    ep_rew_mean        | 1.1e+03   |
| time/                 |           |
|    fps                | 574       |
|    iterations         | 17000     |
|    time_elapsed       | 148       |
|    total_timesteps    | 85000     |
| train/                |           |
|    entropy_loss       | -1.35     |
|    explained_variance | 0.000934  |
|    learning_rate      | 0.0007    |
|    n_updates          | 16999     |
|    policy_loss        | -0.000566 |
|    std                | 0.932     |
|    value_loss         | 2.18e-07  |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 1.1e+03   |
|    ep_rew_mean        | 1.1e+03   |
| time/                 |           |
|    fps                | 574       |
|    iterations         | 17100     |
|    time_elapsed       | 148       |
|    total_t

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 1.1e+03   |
|    ep_rew_mean        | 1.1e+03   |
| time/                 |           |
|    fps                | 574       |
|    iterations         | 18300     |
|    time_elapsed       | 159       |
|    total_timesteps    | 91500     |
| train/                |           |
|    entropy_loss       | -1.33     |
|    explained_variance | 2.91e-05  |
|    learning_rate      | 0.0007    |
|    n_updates          | 18299     |
|    policy_loss        | -0.000617 |
|    std                | 0.92      |
|    value_loss         | 4.51e-07  |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 1.1e+03   |
|    ep_rew_mean        | 1.1e+03   |
| time/                 |           |
|    fps                | 574       |
|    iterations         | 18400     |
|    time_elapsed       | 160       |
|    total_t

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 1.1e+03   |
|    ep_rew_mean        | 1.1e+03   |
| time/                 |           |
|    fps                | 575       |
|    iterations         | 19600     |
|    time_elapsed       | 170       |
|    total_timesteps    | 98000     |
| train/                |           |
|    entropy_loss       | -1.37     |
|    explained_variance | 2.71e-05  |
|    learning_rate      | 0.0007    |
|    n_updates          | 19599     |
|    policy_loss        | -0.000604 |
|    std                | 0.956     |
|    value_loss         | 5.03e-07  |
-------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.1e+03  |
|    ep_rew_mean        | 1.1e+03  |
| time/                 |          |
|    fps                | 575      |
|    iterations         | 19700    |
|    time_elapsed       | 171      |
|    total_timesteps

## Testing the model with rendering

In [20]:
obs = env_rendering.reset()
for i in range(1000):
    action, _state = model.predict(obs, deterministic=False)
    obs, reward, done, info = env_rendering.step(action)
    env_rendering.render()
    if done:
      obs = env_rendering.reset()

In [1]:
env.close()
env_rendering.close()

NameError: name 'env' is not defined